<img style="float: center;" src='https://github.com/STScI-MIRI/MRS-ExampleNB/raw/main/assets/banner1.png' alt="stsci_logo" width="1000px"/> 

# TSO JWebbinar Notebook 2: Exploring TSO Products
-----

**Author**: Néstor Espinoza, AURA Assistant Astronomer, NIRISS branch
<br>
**Last Updated**: November 12, 2021
<br>
**Pipeline Version**: x.x.x

## Table of contents
1. [Introduction](#intro)<br>
   1.1 [Purpose of this Notebook](#purpose)<br>
   1.2 [Input Simulations](#inputs)<br>
   1.3 [Caveats for Simulated Data](#nirisscaveats)<br>

---

1.<font color='white'>-</font>Introduction <a class="anchor" id="intro"></a>
------------------

### 1.1<font color='white'>-</font>Purpose of this Notebook<a class="anchor" id="purpose"></a> ###

In this notebook we provide material to get started on the exploration of TSO products; in particular, we focus on outputs from `detector1` and the `spec2` stages of the pipeline.

### 1.2<font color='white'>-</font>Input Simulations<a class="anchor" id="inputs"></a> ###

The input data for this notebook are simulations made by the NIRISS/SOSS team of a transit of WASP-43b. These simulations were made using the [`mirage`](https://mirage-data-simulator.readthedocs.io/en/latest/) simulator. This dataset can be found [here](https://stsci.app.box.com/folder/135937177294?s=6gegobzzkr8xuptgy0udodob9hq4kvi1).

We will be using products at three different stages of the pipeline, namely:

1. **The Uncalibrated Products**. These correspond to data that have not yet been passed through any of the stages of the pipeline. These have `*uncal.fits` extensions.
2. **The Stage 1 Calibrated Products**. These corrspond to data that have been passed through the `detector1` stage of the pipeline and contain the slopes of the up-the-ramp samples for each integration. These have `*_1_rampfitset.fits` extension.
3. **The partially Stage 2 Calibrated Products**. These correspond to data that have only been calibrated up to the `assign_wcs` step, with which one can obtain the wavelength map of the spectra. These have `*_1_assignwcsstep.fits` extensions.

These products are downloaded directly from this notebook below.

### 1.3<font color='white'>-</font>Caveats for Simulated Data<a class="anchor" id="mirisim"></a> ###

The [`mirage`](https://mirage-data-simulator.readthedocs.io/en/latest/) simulator is not perfect. Be aware that how close these simulations are to real data is not going to be perfectly known until we obtain on-sky data with JWST. As such, a big disclaimer is that while `mirage` simulations represent a large fraction of the systematics that we expect to see, it is in no way a guarantee that this will be the full picture of what we will _actually_ see on real JWST data.

Also note that time-series systematics have not been added to those simulations. This has been important for previous observatories to consider, and appear due to, e.g., thermal breathing of the instruments, focus changes, etc. 

2.<font color='white'>-</font>Setup <a class="anchor" id="setup"></a>
------------------

In this section we set things up a number of necessary things in order for the pipeline to run successfully.

First we'll set the CRDS context; this dictates the versions of various pipeline reference files to use.  Ordinarily you wouldn't want to set a specific version as the latest pipeline should already use the most-recent reference files (and hard-coding a version could get you old reference files that have since been replaced).  However, since this demo is using an old version 1.1.0 of the pipeline, we need to tell it to get some more recent reference files.

Next we'll import the various python packages that we're actually going to use in this notebook, including both generic utility functions and the actual pipeline modules themselves.

Next, we'll specify the data directory structure that we want to use.  In order to keep our filesystem clean we'll separate simulated inputs and outputs from each pipeline stage into their own folders.

Finally, for convenience in this JWebbinar we'll define a flag that sets whether or not to actually run some of the longer pipeline steps in this notebook or just to rely upon cached reductions provided ahead of time.  This is because a number of steps (particularly building 3d data cubes) can take quite a long time to run, and in a short Webbinar we don't want to be waiting for them to all run in real time.  This flag is set to False by default for use in the live Webbinar; if you want to experiment with running all steps yourself ahead of time just set this flag to True.  Total notebook runtime with True/False values is about 1.5 hours vs 3 minutes.


In [ ]:
# Need to set these enviromental variables for this notebook to work properly:
%set_env CRDS_PATH $HOME/crds_cache
%set_env CRDS_SERVER_URL https://jwst-crds.stsci.edu

# Import all useful libraries:
import os
import numpy as np
import matplotlib.pyplot as plt
from numpy.polynomial import chebyshev
from scipy.ndimage import gaussian_filter1d
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from scipy.interpolate import interp1d

from jwst import datamodels
from astropy.io import fits
from astropy.utils.data import download_file

from jwst.pipeline import calwebb_detector1
from jwst.pipeline import calwebb_spec2

3.<font color='white'>-</font>A tour through the `Detector1` stage <a class="anchor" id="setup"></a>
------------------

Here, we will play with one segment of the Uncalibrated data. Let's first download this file:

In [ ]:
# Download uncal file:
file_link = 'https://stsci.box.com/shared/static/dy0t8hza8tzpoqcdlny54e9y6pbsjctc.fits'
filename = 'WASP43_NIS_SOSS-seg002_CLEAR_uncal.fits'

# Look if the file has been downloaded. If not, do it:
if os.path.isdir(filename):
    
    print(filename + ' already exists. Not downloading.')

else:
    
    print('Downloading {}...'.format(filename))
    download_filename = download_file(file_link, cache=True)
    # Rename file:
    os.rename(download_filename, filename)

Let's load the data inside a JWST `datamodel`:

In [ ]:
uncal_data = datamodels.open("WASP43_NIS_SOSS-seg002_CLEAR_uncal.fits")

In [ ]:
# Cover a bug that exists given the simulated data was created a while ago. This will not be 
# needed to be done on real data, but it's a nice excercise showing how easy it is to deal 
# with JWST datamodels:
uncal_data.meta.dither.dither_points = int(uncal_data.meta.dither.dither_points)

In [ ]:
# Let's run the DQ init step --- this will assign DQ flags to our fits uncal file. 
# First, create output directory:
os.mkdir('pipeline_outputs_directory')

# Run step:
dq_results = calwebb_detector1.dq_init_step.DQInitStep.call(uncal_data, 
                                                            output_dir='pipeline_outputs_directory', 
                                                            save_results=True)

Let's run the `saturation` and `linearity` steps: